In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.integrate import odeint
#%matplotlib nbagg
%matplotlib notebook

To predict the occurency of an eclipse we solve the N-body problem for the three bodies Earth, Moom and Sun.

In [3]:
def n_body_diff_eqs(t, r, ms, G=1):
    """
    Returns the differential equations to solve
    t: array-like
        Time interval to obtain the solution of the 
        differential equation.
        
    r: array-like
        Initial conditions of the differential equations.
        The length must be 6*N since for each body there
        must be 3 initial conditions for the position (x,
        y, z) and 3 for the velocity (vx, vy, vz).
        Then, r has the following structure r = [x1, y1,
        z1, vx1, vx2, vx3, x2, y2, z2, vx2, vy2, vz2,...]. 
        
    ms: array-like
        Array of length N containing the masses of the N
        bodies.
        
    G: float
        Universal gravitational constant. Default value is
        set to 1 due to the use of the canonical units. 
    """
    
    #Number of bodies. 
    n = 6
    N = int(len(r)/n)
    
    #Equations
    ecs = np.zeros(len(r))
    
    for i in range(N):
        dvdt_i = [0, 0, 0]
        
        for j in range(N):
            if i != j:
                d = np.sqrt((r[6*j+0]-r[6*i+0])**2 +
                            (r[6*j+1]-r[6*i+1])**2 +
                            (r[6*j+2]-r[6*i+2])**2 )
        
                for k in range(3):
                    #Acceleration dv/dt
                    dvdt_i[k] += G*ms[j]*( r[6*j+k] - r[6*i+k] ) / d**3
                    
        for l in range(3):
            ecs[6*i + l] = r[6*i + 3 + l]
            ecs[6*i + 3 + l] = dvdt_i[l] 
            
    return ecs


def plot_n_body_sol(sol, labels=False, draw_path=True, draw_init=True, draw_end=False, figsize=(9,9)):
    """
    Plots the solution of the N-body problem.
    sol: array-like
        Array of shape (len(ts), 6*N) containing the so-
        lution of the N-body problem.
        
    labels: bool/array-like
        If type is array-like then the array must content
        the names of the N bodies for which the problem 
        was solved.
    
    draw_path: bool
        Draw the path that each particule follows in the 
        time.
    
    draw_init: bool
        Draw the initial position of the N particles.
        
    draw_end: bool
        Draw the last position of the particule in the time
        of the simulation.
        
    figsize: tuple
        tuple containing the dimensions of the plot.
    """
    #Number of bodies
    N = int(sol.shape[1]/6)
    
    fig = plt.figure(figsize=figsize)
    ax = plt.axes(projection="3d")
    font_label = {
            'family': 'serif',
            'weight': 'normal',
            'size': 8}
    
    font_label2 = {
            'family': 'serif',
            'weight': 'normal',
            'size': 10}
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    
    if np.array(labels).any:
        for i in range(N):
            if draw_path: ax.plot(sol.T[6*i+0],        sol.T[6*i+1],     sol.T[6*i+2],    c=colors[i])
            if draw_end:  ax.scatter(sol.T[6*i+0][-1], sol.T[6*i+1][-1], sol.T[6*i+2][-1],c=colors[i], marker="o")
            if draw_init: ax.scatter(sol.T[6*i+0][0],  sol.T[6*i+1][0],  sol.T[6*i+2][0], c=colors[i], 
                                     label=labels[i], marker="o")

        plt.legend(loc="lower center", ncol=len(labels),
                   fancybox=True, shadow=True,
                   mode="expand",prop=font_label)

    else:
        for i in range(N):
            ax.plot(sol.T[6*i+0], sol.T[6*i+1], sol.T[6*i+2])
            ax.scatter(sol.T[6*i+0][0], sol.T[6*i+1][0], sol.T[6*i+2][0], "o")
    
    ax.set_xlabel("X [AU]", fontdict=font_label2)
    ax.set_ylabel("Y [AU]", fontdict=font_label2)
    ax.set_zlabel("Z [AU]", fontdict=font_label2)
    ax.set_zlim(ax.get_xlim())
    plt.tight_layout()

In this notebook we are going to use canonical units. The unit of mass is the mass of the Sun, and we set, as usual, $G=1$. 

In [4]:
#Canonical units
m  = 1
km = 1000*m
kg = 1
s  = 1
G  = 6.67408e-11*m**3/kg/s**2

ul = 1.496e11*m
um = 1.9891e30*kg
ut = np.sqrt(ul**3/(G*um))
G  = 1